In [1]:
# Usefull imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from hmmlearn import hmm
from IOHMM import IOHMM_model

In [2]:
    def _log_likelihood(self, gamma, xi):
        
        l = 0

        zeta = torch.exp(self._compute_log_zeta())
        xi = torch.exp(self._compute_log_xi(alpha=self._forward(), beta=self._backward()))

        log_dnorm_prob = torch.zeros((self.num_states))
        for i in  range(self.num_states):
            log_dnorm_prob[i] = self.log_dnorm(self.outputs[0],
                                               self.emission_matrix[i].dot(torch.cat((torch.tensor([1.0]), self.inputs[0]))),
                                               self.log_sd[i])

        l += torch.sum(zeta[0] * log_dnorm_prob) + torch.sum(torch.exp(xi[0]) * self.log_initial_pi)

        for t in range(1,len(self.outputs)):
            for i in  range(self.num_states):
                
                log_dnorm_prob = self.log_dnorm(self.outputs[t],
                                                self.emission_matrix[i].dot(torch.cat((torch.tensor([1.0]), self.inputs[t]))),
                                                self.log_sd[i])

                l += zeta[t,i] * log_dnorm_prob

                for j in range(self.num_states):
                    l += xi[t,i,j] * self.log_softmax(self.inputs[t])[i,j]

        return l

In [3]:
hmm_model = hmm.GaussianHMM(n_components=3, covariance_type="diag", n_iter=100)

hmm_model.startprob_ = np.array([0., 1., 0.])

hmm_model.transmat_ = np.array([[0.3, 0.3, 1-0.3-0.3],
                                [0.3, 0.3, 1-0.3-0.3],
                                [0.3, 0.3, 1-0.3-0.3]])

hmm_model.means_ = np.array([[.0], [.1], [.2]], )

hmm_model.covars_ = np.array([[.1], [.1], [.1]])


In [4]:
#sample data
X, Z = hmm_model.sample(50)

#dataframe
data = pd.DataFrame(X, columns=['X'])
data['Z'] = Z

data[:]

,X,Z
0,0.389138,1
1,0.246452,2
2,-0.550538,1
3,-0.069431,1
4,0.275763,2
5,-0.165293,2
6,-0.120248,1
7,-0.009135,2
8,0.745523,1
9,0.176876,2


In [5]:
# Torch  data
output = torch.tensor(data['X'].values, dtype=torch.float32)
# Input data ones
input = torch.ones(output.size(), dtype=torch.float32).unsqueeze(1)

In [6]:
IOHMM = IOHMM_model(num_states=3, inputs=input, outputs=output, max_iter=20, tol=1e-4)

In [7]:
IOHMM._baum_welch()

Iteration 1, likelihood: -inf
dim of log_zeta[t-1]: torch.Size([3])
dim of log_softmax(U[t]): torch.Size([3, 3])
dim of log_zeta[t-1]: torch.Size([3])
dim of log_softmax(U[t]): torch.Size([3, 3])
dim of log_zeta[t-1]: torch.Size([3])
dim of log_softmax(U[t]): torch.Size([3, 3])
dim of log_zeta[t-1]: torch.Size([3])
dim of log_softmax(U[t]): torch.Size([3, 3])
dim of log_zeta[t-1]: torch.Size([3])
dim of log_softmax(U[t]): torch.Size([3, 3])
dim of log_zeta[t-1]: torch.Size([3])
dim of log_softmax(U[t]): torch.Size([3, 3])
dim of log_zeta[t-1]: torch.Size([3])
dim of log_softmax(U[t]): torch.Size([3, 3])
dim of log_zeta[t-1]: torch.Size([3])
dim of log_softmax(U[t]): torch.Size([3, 3])
dim of log_zeta[t-1]: torch.Size([3])
dim of log_softmax(U[t]): torch.Size([3, 3])
dim of log_zeta[t-1]: torch.Size([3])
dim of log_softmax(U[t]): torch.Size([3, 3])
dim of log_zeta[t-1]: torch.Size([3])
dim of log_softmax(U[t]): torch.Size([3, 3])
dim of log_zeta[t-1]: torch.Size([3])
dim of log_softmax(

In [14]:
matrix=torch.tensor([[1,2,3],[4,5,6],[7,8,9]])
vector=torch.tensor([1,2,3])
print(vector.shape)
print(matrix.shape)
sum_vector=vector+matrix
print(sum_vector)
torch.sum(sum_vector, dim=0)

torch.Size([3])
torch.Size([3, 3])
tensor([[ 2,  4,  6],
        [ 5,  7,  9],
        [ 8, 10, 12]])


tensor([15, 21, 27])

In [9]:
IOHMM.viterbi()

tensor([0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2,
        0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2,
        0, 2])